In [1]:
import pandas as pd
import numpy as np

In [2]:
patient_df=pd.read_csv('../../pcms_hackathon_data/test/patient.csv')

In [4]:
patient_df['hot_spotter_readmission_flag'] = (
    patient_df['hot_spotter_readmission_flag'] == 't'
).astype(int)

patient_df['hot_spotter_chronic_flag'] = (
    patient_df['hot_spotter_chronic_flag'] == 't'
).astype(int)

patient_df['hot_spotter_identified_at'] = pd.to_datetime(
    patient_df['hot_spotter_identified_at'], errors='coerce'
)


patient_df.loc[
    patient_df['hot_spotter_identified_at'] < '1900-01-01',
    'hot_spotter_identified_at'
] = pd.NaT


patient_df['has_hotspot_flag'] = patient_df['hot_spotter_identified_at'].notna().astype(int)


today = pd.to_datetime('2025-11-01')
patient_df['days_since_hotspot_identified'] = (
    today - patient_df['hot_spotter_identified_at']
).dt.days.fillna(0)


patient_df['age'] = patient_df['age'].fillna(patient_df['age'].median())


max_days = patient_df['days_since_hotspot_identified'].max()
patient_df['hotspot_recency_norm'] = (
    patient_df['days_since_hotspot_identified'] / max_days
).fillna(0)


patient_df['chronic_x_readmission'] = (
    patient_df['hot_spotter_chronic_flag'] * patient_df['hot_spotter_readmission_flag']
)


patient_df['elderly_flag'] = (patient_df['age'] >= 65).astype(int)


train_patient_cols = [
    'patient_id',
    'age',
    'hot_spotter_readmission_flag',
    'hot_spotter_chronic_flag',
    'has_hotspot_flag',
    'days_since_hotspot_identified',
    'hotspot_recency_norm',
    'chronic_x_readmission',
    'elderly_flag'
]

patient_final_test = patient_df[train_patient_cols].copy()

patient_final_test.to_csv('test_patient_final.csv', index=False)
print("Shape:", patient_final_test.shape)
print("Columns:", patient_final_test.columns.tolist())


Shape: (2001, 9)
Columns: ['patient_id', 'age', 'hot_spotter_readmission_flag', 'hot_spotter_chronic_flag', 'has_hotspot_flag', 'days_since_hotspot_identified', 'hotspot_recency_norm', 'chronic_x_readmission', 'elderly_flag']
